<a href="https://colab.research.google.com/github/mveer1/Jupyter-Notebooks/blob/main/Metaclasses%2C_generators%2C_decorators%2C_contextmanagers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Metaclasses**

General purpose of use is when you have 
1. A Base class (library code)
2. A derived class (user code)

In [ ]:
#Library.py
class Base:
    def foo(self):
        return self.bar()

In [ ]:
#user.py
class Derived(Base):
    def bar(self):
        return 'bar'

How do we make sure before actually deploying the code, user always makes a method "bar".

In [ ]:
#library.py
class Base:
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(fun, name, base=None, **kw):
    print("my_bc")
    if base is Base:
        print("check if bar method is defined")
    if base is not None:
        return old_bc(fun, name, base, **kw)
    return old_bc(fun, name, **kw)
print(old_bc, my_bc)
import builtins
builtins.__build_class__ = my_bc

thats ig one way

In [ ]:
#library.py
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if not 'bar' in body:
            raise TypeError("bad user class")
        return super().__new__(cls, name, bases, body)

class Base(metaclass = BaseMeta):
    def foo(self):
        return self.bar()

# **Decorators**

In [ ]:
from time import time
def timer(func):
    def f(x, y=10):
        before = time()
        rv = func(x,y)
        after = time()
        print("elapsed", after-before)
        return rv
    return f

def add(x, y=10):
    return x+y
add = timer(add)

is same as

In [1]:
from time import time
def timer(func):
    def f(x, y=10):
        before = time()
        rv = func(x,y)
        after = time()
        print("elapsed", after-before)
        return rv
    return f

@timer
def add(x, y=10):
    return x+y

A general version of wrapping a function or making a decorator->

In [ ]:
from time import time
def timer(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print("Elapsed", after- before)
        return rv
    return f

# /@timer
# used *args and **kwargs

In [2]:
# another example of a wrapper/decorator, rather a higher decorator
def ntimes(n):
    def inner(f):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                print('running {.__name__}'.format(f))
                rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner

@ntimes(2)
def add(x, y=10):
    return x+y

@ntimes(5)
def sub(x, y=10):
    return x-y

# **Generators**

In [3]:
#whats the difference between add1 and add2 below
def add1(x,y):
    return x+y

class Adder:
    def __call__(self, x,y):
        return x+y
add2 = Adder()

#theres none

In [ ]:
# this method compute gives us the total list after 5secs using 10units of memory
def compute():
    rv = []
    for i in range(10):
        sleep(.5)
        rv.append(i)
    return rv

#this class, on the other hand, gives us the item one by one each .5 secs, and hence not using extra memory if we discard after looking
class Compute():
    def __iter__(self):
        self.last = 0
        return self
    def __next__(self):
        rv = self.last
        self.last += 1
        if self.last >10:
            raise StopIteration
        sleep(.5)
        return rv

In [ ]:
# now what the class did above, can be done using a yield keyword
def compute():
    for i in range(10):
        sleep(.5)
        yield i

#this yield word, gives back i to the user code and also the control of flow, hence when the user wants to resume, it can.
# thats a GENERATOR

#user code
for val in compute():
    print(val)

In [ ]:
#another example for generators
class Api:
    def run_this_first(self):
        first()
    def run_this_second(self):
        second()
    def run_this_third(self):
        third()

#here we want our class Api to be called by user in first second third pattern, otherwise it wont work
#solution to that is here:
def api():
    first()
    yield
    second()
    yield
    third()
#this does not return any value to the user but it gives the control to move forward or not

# **Context Managers**

are about closing the file after opening it. 
file, cursor, anything.

In [11]:
from sqlite3 import connect

with connect("test.db") as conn1:
    cur = conn1.cursor()
    cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
    cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
    cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
    for row in cur.execute("select sum(x*y) from points"):
        print(row)
    cur.execute("drop table points")
    cur.commit()

# thats an example

# general context managers:
    # with ctx() as x:
    #     pass

    # x = ctx.__enter__()
    # try:
    #     pass
    # finally:
    #     x.__exit__()


SyntaxError: ignored

In [12]:
# A context manager in action:
from sqlite3 import connect

class temp:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    def __exit__(self, *args):
        self.cur.execute("drop table points")


with connect("test.db") as conn1:
    cur = conn1.cursor()
    with temp(cur):
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
        for row in cur.execute("select sum(x*y) from points"):
            print(row)
        
    # cur.commit()                                          no need of commit or drop table


In [ ]:
#now we know entry is always before exit, so we can use a generator here
from sqlite3 import connect

def temp(cur):
    cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    print("table created")
    yield
    cur.execute("drop table points")
    print("table dropped")
    

class contextManager:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.gen = temp(self.cur)
        next(self.gen)
    def __exit__(self, *args):
        next(self.gen, None)

with connect("test.db") as conn1:
    cur = conn1.cursor()
    with contextManager(cur):
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
        for row in cur.execute("select sum(x*y) from points"):
            print(row)

In [ ]:
# making a general purpose context manager
from sqlite3 import connect

def temp(cur):
    cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    yield
    cur.execute("drop table points")
    

class contextManager:
    def __init__(self, cur):
        self.cur = cur
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

with connect("test.db") as conn1:
    cur = conn1.cursor()
    with contextManager(temp)(cur):                           #can be changed to, temp = contextmanager(temp)   if put before the with statement, and now that is a place to introduce a decoder statement 
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
        for row in cur.execute("select sum(x*y) from points"):
            print(row)

In [ ]:
#finally
# making a general purpose context manager
from sqlite3 import connect

class contextManager:
    def __init__(self, cur):
        self.cur = cur
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

@contextManager                                             #
def temp(cur):
    cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    yield
    cur.execute("drop table points")

with connect("test.db") as conn1:
    cur = conn1.cursor()
    with temp(cur):                                         #
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
        for row in cur.execute("select sum(x*y) from points"):
            print(row)

In [ ]:
# we can import this contextmanager code from'
from contextlib import contextmanager

In [ ]:
# finally, we get:
from sqlite3 import connect
from contextlib import contextmanager

@contextManager                                             
def temp(cur):
    cur.execute("CREATE TABLE POINTS(X INT, Y INT)")
    try:
        yield
    finally:
        cur.execute("drop table points")

with connect("test.db") as conn1:
    cur = conn1.cursor()
    with temp(cur):                                         
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(2,1)")
        cur.execute("INSERT INTO POINTS (X, Y) VALUES(1,2)")
        for row in cur.execute("select sum(x*y) from points"):
            print(row)